In [4]:
import numpy as np
import sys
import os 
import tensorflow_model_optimization as tfmt 
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt
import random
import zipfile
from tensorflow.keras import Sequential 
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout 
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers 
from tensorflow.keras import layers 
from PIL import Image
from PIL import UnidentifiedImageError
from tqdm import tqdm  
from collections import Counter
from sklearn.preprocessing import LabelEncoder


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
print(tf.__version__)
print("TF MOT version:", tfmt.__version__)
print("TF KERAS", tf.__version__)

In [ ]:

print("Python executable:", sys.executable)            # Ruta del Python en uso
print("TensorFlow version:", tf.__version__)           # Asegúrate sea >=2.9 (ideal 2.10+)
print("TF MOT version:", tfmt.__version__)            # Al menos 0.7.0, preferible 0.8.0
print("Eager mode:", tf.executing_eagerly())           # Debe ser True en TF2

In [ ]:
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))


In [15]:
# Paso 1: Preprocesamiento del Dataset
# Definir el directorio donde se encuentran las imágenes del dataset
with zipfile.ZipFile('/home/pibezx/Documents/CNN/pokemon.zip', 'r') as zip_ref:
    zip_ref.extractall('pokemos')


In [16]:
dataset_dir = "pokemos/PokemonData"

In [ ]:
# Paso 1: Preprocesamiento del Dataset
# Parámetros de preprocesamiento
IMG_SIZE = (224, 224)  # Tamaño al que se redimensionarán las imágenes
BATCH_SIZE = 32  # Tamaño del lote para preprocesamiento

# Listar todas las imágenes en el directorio
image_files = []
labels = []
for root, _, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(('jpg', 'jpeg', 'png')):
            image_files.append(os.path.join(root, file))
            labels.append(os.path.basename(root))

# Convertir listas de archivos y etiquetas a tensores
image_files = tf.constant(image_files)
labels = tf.constant(labels)

# Filtrar clases con al menos 2 ejemplos
label_counts = Counter(labels.numpy())
valid_labels = [label for label, count in label_counts.items() if count >= 2]
filtered_image_files = []
filtered_labels = []
for img, label in zip(image_files.numpy(), labels.numpy()):
    if label in valid_labels:
        filtered_image_files.append(img)
        filtered_labels.append(label)

image_files = tf.constant(filtered_image_files)
labels = tf.constant(filtered_labels)

print(f"Total de imágenes después de filtrar: {len(image_files)}")


# Codificar las etiquetas para que sean valores enteros
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.numpy())

# Convertir etiquetas a tensor de TensorFlow
labels = tf.constant(labels)

# Crear un dataset de TensorFlow con las rutas de las imágenes y etiquetas
def parse_function(filename, label):
    # Leer el archivo de imagen
    img = tf.io.read_file(filename)
    # Decodificar la imagen
    img = tf.image.decode_jpeg(img, channels=3)
    # Redimensionar la imagen
    img = tf.image.resize(img, IMG_SIZE)
    # Normalizar los valores de los píxeles entre 0 y 1
    img = img / 255.0
    return img, label

dataset = tf.data.Dataset.from_tensor_slices((image_files, labels))
dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
#dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
print(f"Número de imágenes cargadas inicialmente: {len(image_files)}")
print(f"Número de etiquetas cargadas inicialmente: {len(labels)}")



In [ ]:
# Paso 2: División del Dataset
print("Dividiendo el dataset en conjuntos de entrenamiento, validación y prueba...")
dataset = dataset.shuffle(len(image_files), reshuffle_each_iteration=False)
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
print(f"Número total de muestras en el dataset: {dataset_size}")
train_size = int(0.7 * len(image_files))
val_size = int(0.2 * len(image_files))
test_size = dataset_size - train_size - val_size

print(f"Tamaño del conjunto de Entrenamiento: {train_size}")
print(f"Tamaño del conjunto de Validación: {val_size}")
print(f"Tamaño del conjunto de Prueba: {test_size}")

train_dataset = dataset.take(train_size)
val_test_dataset = dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)
"""
# Definir la función de data augmentation
def augment(image, label):
    # Aplicar transformaciones aleatorias
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.image.random_saturation(image, lower=0.9, upper=1.1)
    image = tf.image.random_hue(max_delta=0.1)
    # Asegurarse de que la imagen sigue en el rango [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# Aplicar data augmentation solo al conjunto de entrenamiento
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
"""
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"Tamaño del conjunto de Entrenamiento (número de lotes): {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Tamaño del conjunto de Validación (número de lotes): {tf.data.experimental.cardinality(val_dataset).numpy()}")
print(f"Tamaño del conjunto de Prueba (número de lotes): {tf.data.experimental.cardinality(test_dataset).numpy()}")


In [ ]:
# Paso 3: Entrenamiento del Modelo con MobileNetV2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
print("Configurando el modelo MobileNetV2 para entrenamiento...")
#input_tensor = Input(shape=(224, 224, 3))  # Usando 3 canales ya que las imágenes son ahora "RGB"
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# -----------
# Paso 2: (Opcional) Congelar las capas de la base
# -----------
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,               # Base MobileNetV2 podada
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(valid_labels), activation='softmax')
])


# -----------
# Paso 3: Definir parámetros para pruning
# -----------
epochs = 40
steps_per_epoch =  np.ceil(train_size / BATCH_SIZE) # Ajusta según tu dataset

pruning_params = {
    'pruning_schedule': tfmt.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.6,
        begin_step=0,
        end_step=steps_per_epoch * epochs
    )
}

# -----------
# Paso 4: Envolver la base en prune_low_magnitude

print("model type:", type(model))
print("model module:", type(model).__module__)
print("model qualname:", type(model).__qualname__)
# -----------
pruned_model= tfmt.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# -----------
# Paso 6: Compilar
# -----------
pruned_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# -----------
# Paso 7: Entrenar con callbacks para pruning y early stopping
# -----------
callbacks = [
    tfmt.sparsity.keras.UpdatePruningStep(),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
]

history = pruned_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks
)
"""
# Parámetros para pruning
pruning_params = {
    'pruning_schedule': tfmt.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0, 
        final_sparsity=0.6, 
        begin_step=0, 
        end_step=steps_per_epoch * epochs
    )
}
prune_low_magnitude = tfmt.sparsity.keras.prune_low_magnitude

# === Paso 1: Envolver la base de MobileNetV2 con prune_low_magnitude ===
pruned_model_base = prune_low_magnitude(base_model, **pruning_params)

# === Paso 2: Construir la parte final de la red usando la salida de la red podada ===
x = pruned_model_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(valid_labels), activation='softmax')(x)

# === Paso 3: Crear el modelo que realmente usaremos, con pruning incluido ===
pruned_model = Model(inputs=pruned_model_base.input, outputs=predictions)

# Compilar
pruned_model.compile(optimizer=Adam(learning_rate=0.0001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
pruning_callback = tfmt.sparsity.keras.UpdatePruningStep()

# Entrenar
pruned_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[early_stopping, pruning_callback]
)
"""

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = pruned_model.evaluate(test_dataset)

# Imprimir la precisión en el conjunto de prueba
print(f"Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")


In [ ]:
# Paso 1: Obtener las etiquetas verdaderas y predichas
y_true = []
y_pred = []

for images, labels in test_dataset:
    predictions = pruned_model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(predicted_labels)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Obtener los nombres de las clases y decodificarlos si es necesario
class_names = label_encoder.classes_
class_names = [name.decode('utf-8') if isinstance(name, bytes) else name for name in class_names]

# Número máximo de clases a visualizar
max_classes = 20

# Índices de las clases que deseas visualizar (por ejemplo, las primeras 20)
clases_mostrar = np.arange(max_classes)

# Obtener los nombres de las clases de la lista ya decodificada
class_names_mostrar = [class_names[i] for i in clases_mostrar]


# Normalizar la matriz de confusión completa
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Extraer la submatriz para las clases seleccionadas
cm_mostrar = cm_norm[np.ix_(clases_mostrar, clases_mostrar)]

# Visualizar la matriz de confusión normalizada para las clases seleccionadas
plt.figure(figsize=(12, 10))
sns.heatmap(cm_mostrar, annot=False, cmap='Blues', xticklabels=class_names_mostrar, yticklabels=class_names_mostrar)
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión Normalizada (20 Clases Seleccionadas)')
plt.show()

# Generar el reporte de clasificación para las clases seleccionadas
report = classification_report(
    y_true, 
    y_pred, 
    labels=clases_mostrar, 
    target_names=class_names_mostrar
)

print("Reporte de Clasificación para las Clases Seleccionadas:\n")
print(report)

In [ ]:
def mostrar_predicciones_en_prueba(dataset, num_images=6):
    # Crear un iterador del dataset
    dataset_iter = iter(dataset.unbatch().shuffle(1000))

    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        # Obtener la imagen y la etiqueta verdadera
        image, label = next(dataset_iter)
        # Expandir dimensiones para predecir
        image_expanded = tf.expand_dims(image, axis=0)
        # Hacer la predicción
        pred_probs = pruned_model.predict(image_expanded)
        pred_label = np.argmax(pred_probs, axis=1)[0]
        # Obtener los nombres de las clases
        true_class_name = label_encoder.inverse_transform([label.numpy()])[0]
        pred_class_name = label_encoder.inverse_transform([pred_label])[0]
        # Mostrar la imagen con las etiquetas
        plt.subplot(2, 3, i+1)
        plt.imshow(image)
        plt.title(f'Verdadero: {true_class_name}\nPredicción: {pred_class_name}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
# Mostrar predicciones en imágenes del conjunto de prueba
mostrar_predicciones_en_prueba(test_dataset)


In [ ]:
def cargar_y_preprocesar_imagen(ruta_imagen, img_size=(224, 224)):
    """
    Carga una imagen desde ruta_imagen, la decodifica, la redimensiona
    y la normaliza a [0.0, 1.0].
    Devuelve un tf.Tensor con la forma (224, 224, 3).
    """
    # 1. Leer la imagen en binario
    img = tf.io.read_file(ruta_imagen)

    # 2. Decodificar la imagen (asumiendo JPG/PNG con 3 canales)
    img = tf.image.decode_jpeg(img, channels=3)

    # 3. Redimensionar la imagen
    img = tf.image.resize(img, img_size)

    # 4. Normalizar la imagen [0,1]
    img = img / 255.0

    return img


In [ ]:
def predecir_varias_imagenes(rutas_imagenes):
    imgs = []
    for ruta_imagen in rutas_imagenes:
        img = cargar_y_preprocesar_imagen(ruta_imagen)
        imgs.append(img)
    # Crear un batch de imágenes
    imgs_batch = tf.stack(imgs, axis=0)
    # Hacer las predicciones
    pred_probs = pruned_model.predict(imgs_batch)
    pred_labels = np.argmax(pred_probs, axis=1)
    pred_class_names = label_encoder.inverse_transform(pred_labels)
    # Mostrar las imágenes con sus predicciones
    plt.figure(figsize=(15, 10))
    for i in range(len(rutas_imagenes)):
        plt.subplot(2, 3, i+1)
        plt.imshow(imgs[i])
        plt.title(f'Predicción: {pred_class_names[i]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    # Imprimir las predicciones
    for i, ruta_imagen in enumerate(rutas_imagenes):
        print(f'Imagen: {ruta_imagen} - Predicción: {pred_class_names[i]}')


In [ ]:
# Lista de rutas de imágenes
rutas_imagenes = [
    'PokemonTest/14564190248867.png',
    'PokemonTest/800px-Gengar.png',
    'PokemonTest/images.jpg',
    'PokemonTest/1.jpg',
    # Agrega más rutas según necesites
]

# Hacer las predicciones
predecir_varias_imagenes(rutas_imagenes)
